# Support Vector Machines (SVM)
SVM is a supervised learning algorithm used for classification. It can be used to perform binary or multiclass classification. SVM works by finding the best line (or hyperplane in high-dimensional data) that separates the given groups. The objective of SVM learning is to find the best parameters so that the hyperplane separates the data and maximizes the margin 
equation of the best hyperplane is given by $wx^T + b =0$

# References and Further Reading
[Introduction to Support Vector Machines (SVM)](https://www.geeksforgeeks.org/introduction-to-support-vector-machines-svm/)

[Classifying data using Support Vector Machines(SVMs) in Python](https://www.geeksforgeeks.org/classifying-data-using-support-vector-machinessvms-in-python/)

[Huang, Shujun, et al. "Applications of support vector machine (SVM) learning in cancer genomics." Cancer genomics & proteomics 15.1 (2018): 41-51.‏](https://cgp.iiarjournals.org/content/cgp/15/1/41.full.pdf))